conda install openpyxl

In [1]:
import pandas as pd  
from datetime import datetime, timedelta
import re
import os
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words

In [2]:
#กำหนด input file / ชื่อ output file
input='RawDataCSV\Biore UV Bright Face Milk_20240929_180332.csv'
output='Biore UV Bright Face Milk_20240929_180332.csv'

<>:2: SyntaxWarning: invalid escape sequence '\B'
<>:2: SyntaxWarning: invalid escape sequence '\B'
C:\Users\earns\AppData\Local\Temp\ipykernel_22036\938710188.py:2: SyntaxWarning: invalid escape sequence '\B'
  input='RawDataCSV\Biore UV Bright Face Milk_20240929_180332.csv'


In [3]:
def convert_date(date_str, file_date):
    # แปลงวันที่จากชื่อไฟล์
    file_date = datetime.strptime(file_date, '%Y%m%d')
    
    # กรณีที่มีคำว่า 'ก่อน'
    if 'ก่อน' in date_str:
        value, unit = date_str.split()[:2]
        value = int(value)
        
        if unit in ['วินาทีก่อน', 'นาทีก่อน', 'ชั่วโมงก่อน']:
            return file_date.strftime('%Y%m%d')
        elif unit == 'วันก่อน':
            return (file_date - timedelta(days=value)).strftime('%Y-%m-%d')
        elif unit == 'สัปดาห์ก่อน':
            return (file_date - timedelta(weeks=value)).strftime('%Y-%m-%d')
    
    month_dict = {
        'ม.ค.': '01', 'ก.พ.': '02', 'มี.ค.': '03', 'เม.ย.': '04',
        'พ.ค.': '05', 'มิ.ย.': '06', 'ก.ค.': '07', 'ส.ค.': '08',
        'ก.ย.': '09', 'ต.ค.': '10', 'พ.ย.': '11', 'ธ.ค.': '12'
    }
    
    parts = date_str.split()
    if len(parts) == 3 and parts[1] in month_dict:
        day, month, year = parts
        return f"{year}-{month_dict[month]}-{int(day):02d}"
    
    return date_str

def process_dataframe(file_path):
    # อ่านไฟล์ CSV
    df = pd.read_csv(file_path)
    
    # แยกวันที่จากชื่อไฟล์
    file_date = file_path.split('_')[-2]
    
    # แปลงคอลัมน์ date
    df['date'] = df['date'].apply(lambda x: convert_date(x, file_date))
    
    return df


In [4]:
df = process_dataframe(input)
print(df)

      no                        shop          name  rating        date  \
0      1                   BEAUTRIUM         ส***.       5  2021-11-25   
1      2                   BEAUTRIUM         น***.       5  2021-12-11   
2      3                   BEAUTRIUM   ยุพาวรรณ พ.       5  2019-07-15   
3      4                   BEAUTRIUM        Roh C.       5  2020-03-03   
4      5                   BEAUTRIUM  นวลจันทร์ เ.       5  2019-12-16   
..   ...                         ...           ...     ...         ...   
445  446  Kao Beauty & Personal Care          เชลง       5  2024-03-26   
446  447  Kao Beauty & Personal Care         P***.       5  2024-01-07   
447  448  Kao Beauty & Personal Care         0***7       5  2024-07-16   
448  449  Kao Beauty & Personal Care         W***.       5  2024-01-14   
449  450  Kao Beauty & Personal Care         K.🌈🌈🌈       5  2023-12-22   

                                           review_text  
0                               คุณภาพของสินค้าดีมากกก

In [5]:
df.dtypes

no              int64
shop           object
name           object
rating          int64
date           object
review_text    object
dtype: object

In [6]:
# ฟังก์ชันลบข้อมูลที่ไม่ใช่ภาษาไทย
def clean_review(text):
    # ลบอีโมจิ, ภาษาอังกฤษ, สัญลักษณ์พิเศษ, และช่องว่างเกินจำเป็น โดยคงไว้เฉพาะตัวเลขและภาษาไทย
    cleaned_text = re.sub(r'[^\u0E00-\u0E7F0-9\s]', '', text)  # คงภาษาไทยและตัวเลข
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # ลบช่องว่างเกินและช่องว่างหน้าหลัง
    return cleaned_text

# ใช้ฟังก์ชัน clean_review กับคอลัมน์ review_text
df['review_text'] = df['review_text'].apply(lambda x: clean_review(str(x)))

In [7]:
# ลบการขึ้นบรรทัดใหม่ใน review_text และทำให้ข้อความอยู่ในบรรทัดเดียวกัน
df['review_text'] = df['review_text'].apply(lambda x: str(x).replace('\n', ' ').strip())

In [8]:
# โหลดฐานข้อมูลคำพื้นฐาน
word_list = set(thai_words())

def is_meaningful(word):
    # เพิ่มการตรวจสอบคำที่มีรูปแบบพิเศษ
    special_patterns = [
        r'^[ก-ฮ]+$',  # คำภาษาไทยทั่วไป
        r'^[ก-ฮ]+ๆ$',  # คำซ้ำ เช่น ดีๆ เร็วๆ
        r'^[ก-ฮ]+[ุึ][ปบ]$',  # คำลงท้ายด้วย ุป, ึป, ุบ, ึบ เช่น ปุ๊ป, ปั๊บ
        r'^แพ[็ค]$',  # คำว่า แพ็ค หรือ แพค
        r'^[ก-ฮ]{1,2}่[ก-ฮ]$',  # คำสั้นๆ ที่มีวรรณยุกต์ เช่น ค่ะ ไว
        r'^[0-9๐-๙]+$',  # ตัวเลข
        r'^[a-zA-Z]+$',  # คำภาษาอังกฤษ
    ]
    
    return (word in word_list or
            any(re.match(pattern, word) for pattern in special_patterns))

def clean_text(text):
    # แยกคำ
    words = word_tokenize(text)
    
    # แทนที่คำที่ไม่มีความหมายด้วยช่องว่าง
    cleaned_words = [word if is_meaningful(word) else ' ' for word in words]
    
    # รวมคำกลับเป็นประโยค
    cleaned_text = ''.join(cleaned_words)
    
    # ลบช่องว่างซ้ำ
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    # ลบช่องว่างที่ไม่จำเป็นตอนต้นและท้ายข้อความ
    cleaned_text = cleaned_text.strip()
    
    return cleaned_text

In [9]:
df['review_text'] = df['review_text'].apply(clean_text)

In [10]:
# ลบแถวที่ review_text ว่าง (ที่ไม่มีข้อมูลภาษาไทยหรือเป็นช่องว่าง)
df_cleaned = df[df['review_text'].astype(bool)].copy()

In [11]:
df_cleaned

,no,shop,name,rating,date,review_text
0,1,BEAUTRIUM,ส***.,5,2021-11-25,คุณภาพของสินค้าดีมากกก
1,2,BEAUTRIUM,น***.,5,2021-12-11,ได้รับสินค้าครบแล้วนะคะ ตรงตามที่สั่งไปค่ะ คุณ...
2,3,BEAUTRIUM,ยุพาวรรณ พ.,5,2019-07-15,สินค้าใช้ดีอยู่แล้ว รอบนี้ได้ราคาถูกอีก สินค้า...
3,4,BEAUTRIUM,Roh C.,5,2020-03-03,สินค้าดี บรรจุภัณฑ์เรียบร้อยส่งไว
4,5,BEAUTRIUM,นวลจันทร์ เ.,5,2019-12-16,ได้ของเร็วคะ แพ็คอย่างดี
...,...,...,...,...,...,...
439,440,Kao Beauty & Personal Care,ว***.,5,2024-08-14,ให้ผิวที่ดูเป็นธรรมชาติ ง่ายต่อการทาและผสม ให้...
441,442,Kao Beauty & Personal Care,กัลยา เ.,5,2023-08-17,กันแดดคุณภาพดีมาก เนื้อสัมผัสนุ่ม ประสิทธิภาพด...
445,446,Kao Beauty & Personal Care,เชลง,5,2024-03-26,สินค้าช่วงจัดโปร ราคาคือดีมาก1แถม1อีดต่างหาก ก...
446,447,Kao Beauty & Personal Care,P***.,5,2024-01-07,ทางร้านแพ็คสินค้ามาดี จัดส่งรวดเร็ว ได้ของถูกต...


In [12]:
# รีเซ็ตเลขลำดับคอลัมน์ 'no' ใน DataFrame ที่ผ่านการตรวจสอบ
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned['no'] = df_cleaned.index + 1

In [13]:
df_cleaned

,no,shop,name,rating,date,review_text
0,1,BEAUTRIUM,ส***.,5,2021-11-25,คุณภาพของสินค้าดีมากกก
1,2,BEAUTRIUM,น***.,5,2021-12-11,ได้รับสินค้าครบแล้วนะคะ ตรงตามที่สั่งไปค่ะ คุณ...
2,3,BEAUTRIUM,ยุพาวรรณ พ.,5,2019-07-15,สินค้าใช้ดีอยู่แล้ว รอบนี้ได้ราคาถูกอีก สินค้า...
3,4,BEAUTRIUM,Roh C.,5,2020-03-03,สินค้าดี บรรจุภัณฑ์เรียบร้อยส่งไว
4,5,BEAUTRIUM,นวลจันทร์ เ.,5,2019-12-16,ได้ของเร็วคะ แพ็คอย่างดี
...,...,...,...,...,...,...
297,298,Kao Beauty & Personal Care,ว***.,5,2024-08-14,ให้ผิวที่ดูเป็นธรรมชาติ ง่ายต่อการทาและผสม ให้...
298,299,Kao Beauty & Personal Care,กัลยา เ.,5,2023-08-17,กันแดดคุณภาพดีมาก เนื้อสัมผัสนุ่ม ประสิทธิภาพด...
299,300,Kao Beauty & Personal Care,เชลง,5,2024-03-26,สินค้าช่วงจัดโปร ราคาคือดีมาก1แถม1อีดต่างหาก ก...
300,301,Kao Beauty & Personal Care,P***.,5,2024-01-07,ทางร้านแพ็คสินค้ามาดี จัดส่งรวดเร็ว ได้ของถูกต...


In [81]:
if not os.path.exists('CleanDataCSV'):
    os.makedirs('CleanDataCSV')
if not os.path.exists('CleanDataExel'):
    os.makedirs('CleanDataExel')

In [14]:
# บันทึก DataFrame ลงไฟล์ CSV
csv_file = f'CleanDataCSV/{output}'
df_cleaned.to_csv(csv_file, index=False)

# บันทึก DataFrame ลงไฟล์ Excel
excel_file = f'CleanDataExel/{output}.xlsx'
df_cleaned.to_excel(excel_file, index=False)

In [13]:
import re
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words
from pythainlp.spell import correct

# โหลดฐานข้อมูลคำพื้นฐาน
word_list = set(thai_words())

def is_meaningful(word):
    # เพิ่มการตรวจสอบคำที่มีรูปแบบพิเศษ
    special_patterns = [
        r'^[ก-ฮ]+$',  # คำภาษาไทยทั่วไป
        r'^[ก-ฮ]+ๆ$',  # คำซ้ำ เช่น ดีๆ เร็วๆ
        r'^[ก-ฮ]+[ุึ][ปบ]$',  # คำลงท้ายด้วย ุป, ึป, ุบ, ึบ เช่น ปุ๊ป, ปั๊บ
        r'^แพ[็ค]$',  # คำว่า แพ็ค หรือ แพค
        r'^[ก-ฮ]{1,2}่[ก-ฮ]$',  # คำสั้นๆ ที่มีวรรณยุกต์ เช่น ค่ะ ไว
        r'^[0-9๐-๙]+$',  # ตัวเลข
        r'^[a-zA-Z]+$',  # คำภาษาอังกฤษ
    ]
    
    return (word in word_list or
            any(re.match(pattern, word) for pattern in special_patterns))

def remove_repeated_chars(word):
    # ลบตัวอักษรที่ซ้ำกันเกิน 2 ครั้ง
    return re.sub(r'(.)\1{2,}', r'\1\1', word)

def clean_text(text):
    # แยกคำ
    words = word_tokenize(text)

    
    # แทนที่คำที่ไม่มีความหมายด้วยช่องว่าง
    cleaned_words = [word if is_meaningful(word) else ' ' for word in words]
    
    for word in words:
        word = remove_repeated_chars(word)
        if is_meaningful(word):
            cleaned_words.append("")

    # รวมคำกลับเป็นประโยค
    cleaned_text = ''.join(cleaned_words)
    
    # ลบช่องว่างซ้ำ
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    # ลบช่องว่างที่ไม่จำเป็นตอนต้นและท้ายข้อความ
    cleaned_text = cleaned_text.strip()
    
    return cleaned_text

# ตัวอย่างการใช้งาน
examples = [
    "ใช้งานดีมากค่ะ กันแดดเนื้อเจล ปาดปุ๊ปซึมปั๊ปปปปปปป",
    "ส่งไว แพคมาอย่างดี ไม่มีเสียหาย ลองใช้ดูแล้วซึมเร็ว ไม่เหนียวเหมาะสำหรับใช้ในชีวิตประจำวัน",
    "นใำนฝำนฝพนฝกนฝดสฝดนฝะัตงพนฝะนฝพนพนใำนใพนฝำนวำนฝดรใดรใด่มดรมำรมำรใพรใพนใพนฝพนฝพนฝพนฝดนใพน",
    "ดีมากๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆ",
    "ั  ชอบมาก ถูกมาก ดีมาก คุ้มค่า คุ้มราคาสุดๆ",
    "วแ่วแว่แว่อทใ ทใว่แ่วแีวแว่ งาอยีดัวแฝท  าฃอ่วแง่ งา ว่แีววีแวาแ งา ง่แางอางดีงแาง สฃอบรแยีแงาอางอบรเหมาะสำหรับกิจกรรมกลางแจ้ง, เหมาะสำหรับการใช้งานทุกวัน,",
    "ดีค่ะดีค่ะดีค่ะดีค่ะดีค่ะดีค่ะดีค่ะดีค่ะดีค่ะดีค่ะดีค่ะดีค่ะดีค่ะ",
    "ดีมากกกกกกกกกกกกกกกกกกก"
]

for example in examples:
    print("Original:", example)
    print("Cleaned:", clean_text(example))
    print()

# สำหรับการใช้งานกับ DataFrame
# df['cleaned_review'] = df['review_text'].apply(clean_text)
# print(df[['review_text', 'cleaned_review']].head())

Original: ใช้งานดีมากค่ะ กันแดดเนื้อเจล ปาดปุ๊ปซึมปั๊ปปปปปปป
Cleaned: ใช้งานดีมากค่ะ กันแดดเนื้อเจล ปาดปุ๊ปซึมปั๊ปปปปปปป

Original: ส่งไว แพคมาอย่างดี ไม่มีเสียหาย ลองใช้ดูแล้วซึมเร็ว ไม่เหนียวเหมาะสำหรับใช้ในชีวิตประจำวัน
Cleaned: ส่งไว แพคมาอย่างดี ไม่มีเสียหาย ลองใช้ดูแล้วซึมเร็ว ไม่เหนียวเหมาะสำหรับใช้ในชีวิตประจำวัน

Original: นใำนฝำนฝพนฝกนฝดสฝดนฝะัตงพนฝะนฝพนพนใำนใพนฝำนวำนฝดรใดรใด่มดรมำรมำรใพรใพนใพนฝพนฝพนฝพนฝดนใพน
Cleaned: ใด

Original: ดีมากๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆ
Cleaned: ดีมาก

Original: ั  ชอบมาก ถูกมาก ดีมาก คุ้มค่า คุ้มราคาสุดๆ
Cleaned: ชอบมาก ถูกมาก ดีมาก คุ้มค่า คุ้มราคาสุด

Original: วแ่วแว่แว่อทใ ทใว่แ่วแีวแว่ งาอยีดัวแฝท  าฃอ่วแง่ งา ว่แีววีแวาแ งา ง่แางอางดีงแาง สฃอบรแยีแงาอางอบรเหมาะสำหรับกิจกรรมกลางแจ้ง, เหมาะสำหรับการใช้งานทุกวัน,
Cleaned: งาอยี แง่ งา วี งา อางดี แง อางอบรเหมาะสำหรับกิจกรรมกลางแจ้ง เหมาะสำหรับการใช้งานทุกวัน

Original: ดีค่ะดีค่ะดีค่ะดีค่ะดีค่ะดีค่ะดีค่ะดีค่ะดีค่ะดีค่ะดีค่ะดีค่ะดีค่ะ
Cleaned: ดีค

In [93]:
from pythainlp.spell import correct

test = correct("ปุ๊ป")
print(test)

ปุ๊ป


In [44]:
import re

# ข้อความตัวอย่าง
text = "ตัวอย่าง 123 ภาษาไทย 😃 test @#"

# ลบอีโมจิ, ภาษาอังกฤษ, สัญลักษณ์พิเศษ, และช่องว่างเกินจำเป็น โดยคงไว้เฉพาะตัวเลขและภาษาไทย
cleaned_text = re.sub(r'[^\u0E00-\u0E7F0-9\s]', '', text)  # คงภาษาไทยและตัวเลข
cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # ลบช่องว่างเกินและช่องว่างหน้าหลัง

print(cleaned_text)


ตัวอย่าง 123 ภาษาไทย
